In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
plt.style.use("ggplot")
sns.set_context("paper")

In [15]:
df = pd.read_csv("TMP_TEST_analyses/cell_nucleus.csv")

In [ ]:
df

In [ ]:
df["ObjectNumber"].unique()

In [ ]:
plt.figure(dpi=200, figsize=(10, 5))

for object_number in range(1, 15):
    ys = []
    for time in df["ImageNumber"].unique():
        try:
            y = df.query(f"ImageNumber == {time} and ObjectNumber == {object_number}")[
                "AreaShape_Eccentricity"
            ].values[0]
        except IndexError:
            continue
        ys.append(y)
    plt.plot(ys, label=f"Object {object_number}")
plt.title("AreaShape_Eccentricity over time")
plt.ylabel("AreaShape_Eccentricity")
plt.xlabel("Time")
plt.legend(loc="upper right")
plt.show()